In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("health-expenditure.csv")
df

,Year,Operating Expenditure ($m),Development Expenditure ($m),Government Health Expenditure1 ($m),Government Health Expenditure1 (as % of GDP)
0,2006,"1,840",96,"2,009.70",0.9
1,2007,"2,019",185,"2,283.20",0.8
2,2008,"2,379",336,"2,814.10",1.0
3,2009,"2,920",711,"3,745.80",1.3
4,2010,"3,258",485,"3,856.70",1.2
5,2011,"3,489",453,"4,091.50",1.2
6,2012,"4,066",605,"4,837.30",1.3
7,2013,"5,044",723,"5,938.10",1.6
8,2014,"5,872","1,147","7,223.10",1.8
9,2015,"7,520","1,413","8,639.90",2.1
